In [1031]:
import cv2
import imutils
from skimage.filters import threshold_local
# from pyimagesearch.transform import four_point_transform
import numpy as np
import random as rng
from matplotlib import pyplot as plt

In [1032]:
rng.seed(12345)

# read and show the input image
img_path = 'document.jpeg'
img = cv2.imread(img_path)


In [1033]:
ratio = img.shape[0] / 500.0 # find the ratio
input_img = imutils.resize(img, height = 500) # resize the image to 500 


In [1034]:
def get_line_coefficients(p1: tuple, p2: tuple):
  x1, y1 = p1
  x2, y2 = p2

  a = y1 - y2
  b = x2 - x1
  c = x1*y2 - x2*y1

  return (a,b,c)


In [1035]:
def get_intersection_points(coeff_1: tuple, coeff_2: tuple):
  a1, b1, c1 = coeff_1
  a2, b2, c2 = coeff_2

  x = 0 
  y = 0

  x_num = b1 * c2 - b2 * c1
  x_den = a1 * b2 - a2 * b1

  y_num = c1 * a2 - c2 * a1
  y_den = a1 * b2 - a2 * b1

  if x_den != 0:
    x = x_num / x_den
  
  if y_den != 0:
    y = y_num / y_den
  
  return (x, y)
  


In [1036]:
def sort_contours(elem):
    return cv2.arcLength(elem, closed=True)
    
def get_quadrilateral(grayscale: cv2.Mat, output: cv2.Mat):
  convex_hull_mask = np.zeros((grayscale.shape[0], grayscale.shape[1], 3), dtype=np.uint8)


  convex_hull_mask_grayscale = cv2.cvtColor(convex_hull_mask, cv2.COLOR_BGR2GRAY)

  # Find contours
  contours, _ = cv2.findContours(grayscale, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  # get the largest contour by sorting the contours wrt arcLength
  contours = sorted(contours, key=sort_contours, reverse=True)[:1]

  # convex hull object
  hull_list = []
  hull = cv2.convexHull(contours[-1], True)
  hull_list.append(hull)

  cv2.drawContours(convex_hull_mask_grayscale, hull_list, -1, (255,0,0), 2, 8)

  cv2.imshow('Convex Hull Mask', convex_hull_mask_grayscale)
  cv2.waitKey(10)

  # # rho: The resolution parameter rho in pixels.
  # # theta: The resolution of the parameter \theta in radians.
  # # threshold: The minimum number of intersecting points to detecta line.

  lines = cv2.HoughLinesP(image = convex_hull_mask_grayscale, rho = 2, theta = np.pi / 200, minLineLength=200, maxLineGap=0, threshold=40)



  # draw all houghlines
  if lines is not None:
    for line in lines:
      l = line[0]
      cv2.line(output, (l[0], l[1]), (l[2], l[3]), (0,255,0), 2, cv2.LINE_AA )
  

  # if at any point there are exactly four lines forming a quad
  if len(lines) == 4:
    coefficients = []
    for line in lines:
      l = line[0]
      coefficients.append(get_line_coefficients((l[0], l[1]), (l[2], l[3])))
  
    corners = []

    rows, cols = grayscale.shape

    # for every combination of coeffs get the intersection points
    for i in range(len(coefficients)):
      for j in range(i, len(coefficients)):

        int_point = get_intersection_points(coefficients[i], coefficients[j])

        x, y = int_point

        # coords should be within the image boundaries
        if x > 0 and y > 0 and x < cols and y < rows:
          cv2.circle(output, (int(x), int(y)),3, (0, 0, 255), 4) 
          corners.append(int_point)
        
    print("corners: ", corners)


  cv2.imshow('Lines', output)
  cv2.waitKey(10)

  



In [1037]:
# Convert image to gray and blur it
input_grey = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
input_grey = cv2.GaussianBlur(input_grey, (3,3), 0)
ret, input_grey = cv2.threshold(input_grey, 130, 200, cv2.THRESH_BINARY)

edges = cv2.Canny(input_grey, 83, 300)

cv2.imshow('Canny', edges)
cv2.waitKey(10)

corners = get_quadrilateral(edges, input_img)


corners:  [(302.82718046771583, 105.01088395352258), (70.4431654676259, 101.95319954562666), (362.9244585314316, 448.9719009564914), (31.33740541205628, 459.0505347899071)]


In [ ]:
cnts,_ = cv2.findContours(edged_img.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE) # find all the contours
print(cnts)

# cv2.drawContours(img, cnts, -1, (0, 255, 0), 3)
  
# cv2.imshow('Contours', img)
# cv2.waitKey(5000)

# Sorting the contours in descending order based on their contour area and just taking the first 5
cnts = sorted(cnts,key=cv2.contourArea,reverse=True)

c = cnts[0]
peri = cv2.arcLength(c,True)
approx = cv2.approxPolyDP(c,0.03*peri,True)
x, y, w, h = cv2.boundingRect(c)
print(x,y,w,h)
to_show = img.copy()
# draw a green rectangle to visualize the bounding rect
cv2.rectangle(to_show, (x, y), (x+w, y+h), (0, 255, 0), 2)
plt.imshow(to_show)
plt.title("contour")
plt.show()

# Traversing in contours and finding the contour with 4 sides using cv2.approxPolyDP().
# for c in cnts:
#     peri = cv2.arcLength(c,True)
#     approx = cv2.approxPolyDP(c,0.096*peri,True)
#     x, y, w, h = cv2.boundingRect(c)
#     # draw a green rectangle to visualize the bounding rect
#     cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
#     plt.imshow(img)
#     plt.title("soemthing")
#     plt.show()
#     print(len(approx), cv2.contourArea(c))
#     if len(approx)==4:
#         doc = approx
#         break

In [ ]:
len(cnts)

In [ ]:
# draw points/circles around the corners of the document. 
p=[]
for d in doc:
    tuple_point = tuple(d[0])
    cv2.circle(img,tuple_point,3,(0,0,255),4)
    p.append(tuple_point)
    
cv2.imshow('Corner points detected',img)
cv2.waitKey(0)

In [ ]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [ ]:
# apply four point transform to extract the document rectangle
warped = four_point_transform(org, doc.reshape(4, 2) * ratio)
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
cv2.imshow("Warped", imutils.resize(warped, height = 650))
cv2.waitKey(0)

In [ ]:
# give a black and white feel to the image
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)
cv2.destroyAllWindows()